In [61]:
import pandas as pd
import seaborn as sns
import numpy as np
import hvplot.pandas
	
import warnings
warnings.filterwarnings('ignore')

In [62]:
#load dataset 
train = pd.read_csv("D:/ZENIUS/DATASET/House Price/train.csv")
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [63]:
# train.info()
train['LotArea'].isnull().sum()

0

In [64]:
#target feature menggunakan sale price

y = train['SalePrice']
X = train[['LotArea','OverallQual','OverallCond']]

# Y_train
X

,LotArea,OverallQual,OverallCond
0,8450,7,5
1,9600,6,8
2,11250,7,5
3,9550,7,5
4,14260,8,5
...,...,...,...
1455,7917,6,5
1456,13175,6,6
1457,9042,7,9
1458,9717,5,6


# Train Test Split

In [65]:
#split data menjadi X_train,X_test,y_train,y_test
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
X_test

,LotArea,OverallQual,OverallCond
892,8414,6,8
1105,12256,8,5
413,8960,5,6
522,5000,6,7
1036,12898,9,5
...,...,...,...
331,8176,5,6
323,5820,3,8
650,8125,7,6
439,12354,6,8


In [66]:
# Eval function
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)
    print('__________________________________')
    
def evaluate(true, predicted):
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    return mae, mse, rmse, r2_square

In [67]:
#Menggunakan preprocessing standardScaler dan menggunakan pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('std_scalar', StandardScaler())
])

X_train = pipeline.fit_transform(X_train)
X_test = pipeline.transform(X_test)

In [68]:
#Memilih model Linier Regresi
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train,y_train)

LinearRegression()

In [69]:
#Membuat dataframe coeficient
coeff_df = pd.DataFrame(lin_reg.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
LotArea,14397.498997
OverallQual,59322.101334
OverallCond,-1346.760727


In [70]:
#melakukan prediksi
pred = lin_reg.predict(X_test)

In [71]:
#Memvisualisasi niali true value dan predicted value
pd.DataFrame({'True Values': y_test, 'Predicted Values': pred}).hvplot.scatter(x='True Values', y='Predicted Values')

:Scatter   [True Values]   (Predicted Values)

In [72]:
#memvisualisasi error values 
pd.DataFrame({'Error Values': (y_test - pred)}).hvplot.kde()

:Distribution   [Error Values]   (Density)

In [73]:
## EVALUASI score machine learning menggunakan MAE,MSE,RMSE,R2 SQUARE
test_pred = lin_reg.predict(X_test)
train_pred = lin_reg.predict(X_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)

results_df = pd.DataFrame(data=[["Linear Regression", *evaluate(y_test, test_pred) , cross_val(LinearRegression())]], 
                          columns=['Model', 'MAE', 'MSE', 'RMSE', 'R2 Square', "Cross Validation"])

Test set evaluation:
_____________________________________
MAE: 31192.052928484627
MSE: 2290184494.818759
RMSE: 47855.872103836526
R2 Square 0.6718037083940698
__________________________________
Train set evaluation:
_____________________________________
MAE: 31940.65035806197
MSE: 2106046264.7305384
RMSE: 45891.67968957487
R2 Square 0.6500756240477343
__________________________________


In [74]:
#Memasukan kedalam csv untuk melihat hasil data
test = pd.read_csv('D:/ZENIUS/DATASET/House Price/test.csv')
X_test = test[['LotArea', 'OverallQual','OverallCond']]
y_pred = lin_reg.predict(X_test)
selesai = pd.DataFrame({'Id': test.Id, 'SalePrice': y_pred})
selesai.to_csv('selesai.csv', index=False)
selesai.head(5)

,Id,SalePrice
0,1461,1.677976e+08
1,1462,2.059383e+08
2,1463,1.995886e+08
3,1464,1.441874e+08
4,1465,7.270864e+07
